In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

In [19]:
import py_entitymatching as em
from py_entitymatching.utils.generic_helper import list_diff
from py_entitymatching.matcherselector.mlmatcherselection import get_xy_data

time: 1.36 ms


In [10]:
from sklearn.cross_validation import KFold, cross_val_score

time: 1.85 ms


In [15]:
A = em.load_dataset('person_table_A', key='ID')
B = em.load_dataset('person_table_B', key='ID')

L = em.read_csv_metadata('person_labeled_data.csv', key='_id')

A1 = A[list_diff(list(A.columns), [em.get_key(A)])]
B1 = B[list_diff(list(B.columns), [em.get_key(B)])]

feature_table = em.get_features_for_matching(A1, B1)

em.set_fk_ltable(L, 'ltable_ID')
em.set_fk_rtable(L, 'rtable_ID')
em.set_ltable(L, A)
em.set_rtable(L, B)

True

time: 30.1 ms


In [48]:
L.ix[2, 'label'] = 0

time: 1.45 ms


In [49]:
L

,_id,ltable_ID,rtable_ID,label
0,0,a2,b3,1
1,1,a3,b2,1
2,2,a5,b5,0
3,3,a1,b1,0
4,4,a2,b2,0
5,5,a2,b4,0
6,6,a2,b5,0
7,7,a3,b1,0
8,8,a4,b4,0
9,9,a5,b4,0


time: 8.33 ms


In [50]:
k=9

E = em.extract_feature_vecs(L, feature_table=feature_table, attrs_after='label', show_progress=False)

cv = KFold(len(L), k, shuffle=True, random_state=0)

time: 35.7 ms


In [51]:
rf = em.RFMatcher()
list_test = []
for train_index, prediction_index in cv:
    train = E.iloc[train_index]
    test = E.iloc[prediction_index]
    ltest = L.iloc[prediction_index]
    rf.fit(table=train, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label') 
    predictions = rf.predict(table=test, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                             target_attr='predicted')
    x,y=get_xy_data(x=None, y=None, table=test, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label') 
    predict_probas = rf.clf.predict_proba(x)
    ltest = ltest.copy()
    ltest['predicted'] = predictions
    ltest['pred_proba_nonmatch'] = predict_probas[:,0]
    ltest['pred_proba_match'] = predict_probas[:,1]
    list_test.append(ltest)
    
    
    

time: 175 ms


In [37]:
import pandas as pd

time: 1.07 ms


In [53]:
pp = pd.concat(list_test)

time: 4.54 ms


In [55]:
pp['conf'] = -1

time: 1.21 ms


In [63]:
def fn(t):
    if t['label'] == 0 and t['predicted'] == 1:
        t['conf'] = t['pred_proba_match']
    elif t['label'] == 1 and t['predicted'] == 0:
        t['conf'] = t['pred_proba_nonmatch']
    return t

time: 2.47 ms


In [64]:
pp

,_id,ltable_ID,rtable_ID,label,predicted,pred_proba_nonmatch,pred_proba_match,conf
2,2,a5,b5,0,1,0.1,0.9,-1
8,8,a4,b4,0,0,1.0,0.0,-1
4,4,a2,b2,0,0,1.0,0.0,-1
9,9,a5,b4,0,0,1.0,0.0,-1
1,1,a3,b2,1,0,0.5,0.5,-1
6,6,a2,b5,0,0,1.0,0.0,-1
7,7,a3,b1,0,0,1.0,0.0,-1
3,3,a1,b1,0,0,1.0,0.0,-1
0,0,a2,b3,1,1,0.4,0.6,-1
5,5,a2,b4,0,0,1.0,0.0,-1


time: 13.1 ms


In [66]:
pp1 = pp.apply(fn, axis=1)

time: 4.76 ms


In [67]:
em.filter_rows(pp1, 'conf != -1')

,_id,ltable_ID,rtable_ID,label,predicted,pred_proba_nonmatch,pred_proba_match,conf
2,2,a5,b5,0,1,0.1,0.9,0.9
1,1,a3,b2,1,0,0.5,0.5,0.5


time: 15.1 ms


In [69]:
pp1.insert(0, 'temp', -1)

time: 3.04 ms


In [70]:
pp1

,temp,_id,ltable_ID,rtable_ID,label,predicted,pred_proba_nonmatch,pred_proba_match,conf
2,-1,2,a5,b5,0,1,0.1,0.9,0.9
8,-1,8,a4,b4,0,0,1.0,0.0,-1.0
4,-1,4,a2,b2,0,0,1.0,0.0,-1.0
9,-1,9,a5,b4,0,0,1.0,0.0,-1.0
1,-1,1,a3,b2,1,0,0.5,0.5,0.5
6,-1,6,a2,b5,0,0,1.0,0.0,-1.0
7,-1,7,a3,b1,0,0,1.0,0.0,-1.0
3,-1,3,a1,b1,0,0,1.0,0.0,-1.0
0,-1,0,a2,b3,1,1,0.4,0.6,-1.0
5,-1,5,a2,b4,0,0,1.0,0.0,-1.0


time: 14.1 ms
